## 랭체인이란 ? 
언어 모델에 기반한 AI 애플리케이션을 쉽게 개발할 수 있도록 도와주는 프레임워크 

기존에는 오픈 AI와 같은 언어 모델의 API를 사용해 원하는 기능을 구현하려면 모든 코드를 작성해야 했다. 랭체인을 사용하면, 손쉽게 요약, 검색 , 질의응답 등 여러 기능을 손쉽게 구현할 수 있다. 



In [9]:
!pip install langchain
!pip install langchain_openai


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
## 랭체인으로 멀티턴 대화하기 

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage,AIMessage

llm = ChatOpenAI(model="gpt-4o")

messages = [
    SystemMessage(content="You are a helpful assistant."),
]

while True:
    user_input = input("User : " )
    if user_input in ["exit", "quit"]:
        print("Exiting the chat. Goodbye!")
        break
    messages.append(HumanMessage(content=user_input))
    response = llm.invoke(messages)
    messages.append(response)
    print("Assistant :", response.content)


Assistant : 안녕하세요! 어떻게 도와드릴까요?
Assistant : 박원진 님, 만나서 반갑습니다! 어떻게 도와드릴까요?
Assistant : 당신이 말씀하시길 본인의 이름은 박원진이라고 하셨습니다. 다른 질문이 있으시면 말씀해 주세요!
Exiting the chat. Goodbye!


In [26]:
## 메시지 히스토리 이해하기 

'''
앞에서는 멀티턴 대화를 위해 매번 GPT와 사용자의 대화 내용을 리스트나 딕셔너리에 추가하는 코드를 작성해야했다. 
랭체인의 메시지 히스토리 기능을 사용하면 멀티턴 대화를 더 쉽게 구축할 수 있다. 
'''

# 메모리 내에서 메시지를 리스트형태로 보관한다. 
# 애플리케이션을 종료하면 대화 내용이 사라지므로, 계속 사용하고 싶다면 파일이나 DB에 저장해야한다. 
from langchain_core.chat_history import InMemoryChatMessageHistory
# 모델을 생성할 때 대화 기록을 함께 전달하는 클래스 
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4o")
# session _id를 기준으로 대화 기록을 저장하는 딕셔너리 
store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_messages_hisotry = RunnableWithMessageHistory(
    model,
    get_chat_history,
)

In [28]:
config = {'configurable': {'session_id': 'user_1234'}}

response = with_messages_hisotry.invoke([HumanMessage(content="Hello!")], config=config)
print(response.content)

c:\PRO\python\ai-agent\AI-Agent-Dev\venv\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


Hello! How can I assist you today?


In [ ]:
## 위 코드 구조를 하나씩 까보면 복잡하기 때문에
# 아래의 간단한 코드 구조로 대충 이해하기.


class SimpleHistoryWrapper:
    def __init__(self, model, get_history):
        self.model = model
        self.get_history = get_history

    def invoke(self, messages, config):
        session_id = config["configurable"]["session_id"]
        history = self.get_history(session_id)

        past = history.messages
        final_input = past + messages

        resp = self.model.invoke(final_input)
        history.add_messages(messages)
        history.add_message(resp)
        return resp

In [32]:
## 스트림 방식으로 출력하기

"""
.invoke라고 되어있던 부분을 .stram으로 바꾸면 스트림 방식으로 출력할 수 있다.
"""

## 메시지 히스토리 이해하기

"""
앞에서는 멀티턴 대화를 위해 매번 GPT와 사용자의 대화 내용을 리스트나 딕셔너리에 추가하는 코드를 작성해야했다. 
랭체인의 메시지 히스토리 기능을 사용하면 멀티턴 대화를 더 쉽게 구축할 수 있다. 
"""

# 메모리 내에서 메시지를 리스트형태로 보관한다.
# 애플리케이션을 종료하면 대화 내용이 사라지므로, 계속 사용하고 싶다면 파일이나 DB에 저장해야한다.
from langchain_core.chat_history import InMemoryChatMessageHistory

# 모델을 생성할 때 대화 기록을 함께 전달하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4o")
# session _id를 기준으로 대화 기록을 저장하는 딕셔너리
store = {}


def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_messages_hisotry = RunnableWithMessageHistory(
    model,
    get_chat_history,
)

config = {'configurable': {'session_id': 'user_1234'}}
for r in with_messages_hisotry.stream([HumanMessage(content="내가 어느 나라 사람인지 맞추어보고, 그 나라의 국가를 불러줘")], config=config):
    print(r.content, end="", flush=True)

당신이 어느 나라 사람인지를 제가 직접적으로 맞출 수는 없습니다. 하지만 대화를 통해 힌트를 준다면 유추할 수 있을지도 모르겠습니다. 예를 들어, 사용하는 언어나 문화적 배경 등에 관한 정보를 주신다면 도움이 될 것입니다.

또한 대한민국의 국가인 애국가의 첫 소절은 "동해물과 백두산이 마르고 닳도록"입니다. 다른 나라에 대해서 알고 싶으시다면 더 말씀해 주세요.

## LCEL로 체인만들기 

LCEL은 랭체인에서 복잡한 작업 흐름을 간단하기 만들고 관리할 수 있도록 돕는 도구이다. 랭체인에서는 이런 작업 흐름을 연결하는 것을 체인이라고 한다. 

In [1]:
from langchain_openai import ChatOpenAI
# 랭체인에서 제공하는 출력 파서
# 참고로 출력파서는 언어 모델이 반환하는 결과에서 원하는 형식의 데이터를 추출하는 도구
# StrOutputParser는 텍스트만 추출하여 반환하며, 다른 파서들은 JSON, 숫자 등 특정 형식을 처리할 수 있다.
from langchain_core.output_parsers import StrOutputParser
model = ChatOpenAI(model="gpt-4o")
from langchain_core.messages import HumanMessage, SystemMessage,AIMessage

messages = [
    SystemMessage(content="너는 미녀와 야스에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화해줘."),
    HumanMessage(content='안녕?  저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?'),
]
parser = StrOutputParser()

# 기존에는 gpt 모델에서 결과를 얻고, 그 결과를 파서에 전달해 텍스트만 추출하는 2단계로 사용했다.
# result = model.invoke(messages)
# parser.invoke(result)

chain = model | parser
chain.invoke(messages)

'안녕하세요, 개스톤. 당신의 초대에 감사드려요, 하지만 오늘은 마을에 가서 아버지 벨을 도와드려야 해요. 아시다시피, 그의 발명품 때문에 항상 무언가 처리할 일이 있거든요. 다음에 다른 기회에 함께 할 수 있으면 좋겠어요.'

In [2]:
## 프롬프트 템플릿 이용하기 

from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

system_template = "너는 {story}에 나오는 {character_a}야. 그 캐릭터에 맞게 사용자와 대화해줘."
human_template = "안녕  저는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity} 같이 할까요?"

prompt_template = ChatPromptTemplate([
    ('system', system_template),
    ('human', human_template)
])

result = prompt_template.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "개스톤",
    "activity": "저녁 식사"
})

print(result)

messages=[SystemMessage(content='너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화해줘.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕  저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 식사 같이 할까요?', additional_kwargs={}, response_metadata={})]


In [3]:
# 랭체인의 | 로 체인 구성하기 

chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "개스톤",
    "activity": "저녁 식사"
})

'안녕하세요, 개스톤. 저녁 초대는 고맙지만 오늘은 혼자 책을 읽으면서 시간을 보내고 싶어요. 당신의 열정은 가상하지만, 저는 제 자신만의 시간도 중요하게 생각해요. 이해해 줄 수 있겠죠?'